In [61]:
from datasets import load_dataset

import pandas as pd
import numpy as np 
import json
import os
import re

In [4]:
ds = ["RAMEDIS", "MME", "HMS", "LIRICAL", "PUMCH_ADM"]

#df = pd.DataFrame()

datasets = {}




for dataset in ds:
    data = load_dataset('chenxz/RareBench', dataset, split = "test")
    datasets[dataset] = data
    print(dataset)
    print(data)

Found cached dataset rare_bench (/Users/juliev/.cache/huggingface/datasets/chenxz___rare_bench/RAMEDIS/1.0.0/7aec8acd7a69695cc23c75157e278fdc1f12c20f4eee0fa8c8bfdc689cdf8b9a)


RAMEDIS
Dataset({
    features: ['Phenotype', 'RareDisease', 'Department'],
    num_rows: 624
})


Found cached dataset rare_bench (/Users/juliev/.cache/huggingface/datasets/chenxz___rare_bench/MME/1.0.0/7aec8acd7a69695cc23c75157e278fdc1f12c20f4eee0fa8c8bfdc689cdf8b9a)


MME
Dataset({
    features: ['Phenotype', 'RareDisease', 'Department'],
    num_rows: 40
})


Found cached dataset rare_bench (/Users/juliev/.cache/huggingface/datasets/chenxz___rare_bench/HMS/1.0.0/7aec8acd7a69695cc23c75157e278fdc1f12c20f4eee0fa8c8bfdc689cdf8b9a)


HMS
Dataset({
    features: ['Phenotype', 'RareDisease', 'Department'],
    num_rows: 88
})


Found cached dataset rare_bench (/Users/juliev/.cache/huggingface/datasets/chenxz___rare_bench/LIRICAL/1.0.0/7aec8acd7a69695cc23c75157e278fdc1f12c20f4eee0fa8c8bfdc689cdf8b9a)


LIRICAL
Dataset({
    features: ['Phenotype', 'RareDisease', 'Department'],
    num_rows: 370
})


Found cached dataset rare_bench (/Users/juliev/.cache/huggingface/datasets/chenxz___rare_bench/PUMCH_ADM/1.0.0/7aec8acd7a69695cc23c75157e278fdc1f12c20f4eee0fa8c8bfdc689cdf8b9a)


PUMCH_ADM
Dataset({
    features: ['Phenotype', 'RareDisease', 'Department'],
    num_rows: 75
})


In [32]:
datasets['HMS'][0]

{'Phenotype': ['HP:0001225',
  'HP:0001382',
  'HP:0001443',
  'HP:0001894',
  'HP:0001974',
  'HP:0003419',
  'HP:0025238',
  'HP:0030833',
  'HP:0030834',
  'HP:0030835',
  'HP:0030836',
  'HP:0030839'],
 'RareDisease': ['OMIM:106300'],
 'Department': None}

In [6]:
datasets['RAMEDIS'][1]

{'Phenotype': ['HP:0001270',
  'HP:0001290',
  'HP:0001522',
  'HP:0002090',
  'HP:0040156'],
 'RareDisease': ['OMIM:271980', 'ORPHA:22'],
 'Department': None}

In [65]:
def get_symptoms_formatted(patient):
    symptom_list = patient['Phenotype']
    phenotype_mapping = json.load(open("mapping/phenotype_mapping.json", "r", encoding="utf-8-sig"))
    sym_word_list = []

    for i in symptom_list:
        if i in phenotype_mapping:
            sym_word_list.append(phenotype_mapping[i])
    
    return (", ").join(sym_word_list)
        


def get_diseases_formatted(patient):
    disease_list = patient['RareDisease']
    disease_mapping = json.load(open("mapping/disease_mapping.json", "r", encoding="utf-8-sig"))
    disease_word_list = []

    for i in disease_list:
        if i in disease_mapping:
            disease_word_list.append(disease_mapping[i])

    #remove chinese characters
    disease_word_list = [word for word in disease_word_list if not re.findall("[^\u0000-\u05C0\u2100-\u214F]+", word)]
    
    return (", ").join(disease_word_list)

    

In [66]:
get_symptoms_formatted(datasets['RAMEDIS'][1])

'Motor delay, Generalized hypotonia, Death in infancy, Pneumonia, Elevated urinary carboxylic acid'

In [67]:
get_diseases_formatted(datasets['RAMEDIS'][0])

'Vitamin B12-unresponsive methylmalonic acidemia/Methylmalonic aciduria due to methylmalonyl-coa mutase deficiency, Vitamin B12-unresponsive methylmalonic acidemia/Methylmalonic aciduria due to methylmalonyl-coa mutase deficiency'

In [81]:
ramedis_diseases = [get_diseases_formatted(datasets['RAMEDIS'][i]) for i in range(len(datasets['RAMEDIS']))]

In [82]:
ramedis_diseases[1]

'Succinic semialdehyde dehydrogenase deficiency/Succinic semialdehyde dehydrogenase deficiency, Succinic semialdehyde dehydrogenase deficiency/Succinic semialdehyde dehydrogenase deficiency'

In [83]:
pd.Series(ramedis_diseases).value_counts()

                                                                                                                                                                                                                                                                                                                                                                357
Vitamin B12-unresponsive methylmalonic acidemia/Methylmalonic aciduria due to methylmalonyl-coa mutase deficiency, Vitamin B12-unresponsive methylmalonic acidemia/Methylmalonic aciduria due to methylmalonyl-coa mutase deficiency                                                                                                                             32
PMM2-CDG/Congenital disorder of glycosylation, type Ia, PMM2-CDG/Congenital disorder of glycosylation, type Ia                                                                                                                                                                                  

In [72]:
mme_diseases = [get_diseases_formatted(datasets['MME'][i]) for i in range(len(datasets['MME']))]

In [89]:
lirical_diseases = [get_diseases_formatted(datasets['LIRICAL'][i]) for i in range(len(datasets['MME']))]

In [75]:
pd.Series(mme_diseases).value_counts()

Cerebrocostomandibular syndrome/Cerebrocostomandibular syndrome, Cerebrocostomandibular syndrome/Cerebrocostomandibular syndrome                                                                                                                                                                                                                                                                                                                        11
Alacrimia-choreoathetosis-liver dysfunction syndrome/Congenital disorder of deglycosylation 1, Alacrimia-choreoathetosis-liver dysfunction syndrome/Congenital disorder of deglycosylation 1                                                                                                                                                                                                                                                             8
Ataxia-intellectual disability-oculomotor apraxia-cerebellar cysts syndrome/Poretti-Boltshauser syndrome          

In [95]:
pd.Series(ramedis_diseases).str.contains("Hypo").sum()

1

In [94]:
pd.Series(lirical_diseases)[pd.Series(lirical_diseases).str.contains("Ehlers")==True]

11              Ehlers-Danlos syndrome, classic type, 2
22    Classical-like Ehlers-Danlos syndrome type 2/E...
dtype: object